#**npy Generator**


In [ ]:
!pip install patchify

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from patchify import patchify
import numpy as np
import os
import datetime
import sys

In [ ]:

PATCH_SIZE = (256,256)
NUM_CHANNELS = 3

image_dir="/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/augmented_img"
mask_dir="/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/augmented_msk"
patches_dir="/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/857x21"

In [ ]:
from tqdm.notebook import tqdm  # Add this import if not already present


# Function to create and save patches with tqdm
def create_and_save_patches(image_dir, mask_dir, patch_size=PATCH_SIZE, img_size=PATCH_SIZE):
    if not os.path.exists(patches_dir):
        os.makedirs(os.path.join(patches_dir, 'images'))
        os.makedirs(os.path.join(patches_dir, 'masks'))

    image_files = sorted([os.path.join(image_dir, fname) for fname in os.listdir(image_dir)])
    mask_files = sorted([os.path.join(mask_dir, fname) for fname in os.listdir(mask_dir)])

    for idx, (img_file, mask_file) in tqdm(enumerate(zip(image_files, mask_files)), total=len(image_files), desc="Saving patches"):
        image = load_img(img_file, target_size=(img_size[0], img_size[1]), color_mode="rgb")
        image = img_to_array(image) / 255.0

        mask = load_img(mask_file, target_size=(img_size[0], img_size[1]), color_mode="grayscale")
        mask = img_to_array(mask) / 255.0
        mask = (mask > 0.5).astype(np.float32)

        img_patches = patchify(image, (patch_size[0], patch_size[1], NUM_CHANNELS), step=(patch_size[0], patch_size[1], NUM_CHANNELS))
        mask_patches = patchify(mask, (patch_size[0], patch_size[1], 1), step=(patch_size[0], patch_size[1], 1))

        for i in range(img_patches.shape[0]):
            for j in range(img_patches.shape[1]):
                img_patch = img_patches[i, j, 0]
                mask_patch = mask_patches[i, j, 0]

                np.save(os.path.join(patches_dir, 'images', f"img_{idx}_{i}_{j}.npy"), img_patch)
                np.save(os.path.join(patches_dir, 'masks', f"mask_{idx}_{i}_{j}.npy"), mask_patch)

create_and_save_patches(image_dir, mask_dir)

In [ ]:
import os

def count_files_in_folder(folder_path):
    """Count all files in a directory (excluding subdirectories)"""
    if not os.path.exists(folder_path):
        return f"Directory '{folder_path}' does not exist"

    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return len(files)

# Example usage

path_lst=["/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/857x21/images", "/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/857x21/masks"
          ]

for item in path_lst:
  folder_path = item  # Change this to your folder path
  print(f"Number of files: {count_files_in_folder(folder_path)}")

Number of files: 98
Number of files: 98
